# RNA trans-effects with t tests for 8q

Here we look for genes OUTSIDE the event discussed that are affected by the arm-level event. We will find these effects by performing a series of t-tests comparing the transcriptomics values of the patients with the event against those without the event.

## Setup (Install necessary packages)

We will start by importing necessary packages and collecting all of the transcriptomics data we will need to run the tests. The cancer types analyzed should have been determined in 01_event_basic_info where we determine which types of cancer seem to have the event we are looking at.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cptac
from scipy import stats
import cnvutils
import cptac.utils

In [2]:
# These variables specify which chromosome and arm we're working on, and whether to do cis or trans
CHROMOSOME = '8'
ARM = 'q'
CIS_OR_TRANS = "trans"

In [3]:
if ARM == "p":
    EVENT_COLUMN = "loss_event"
    EXCLUDE_COLUMN = "gain_event"
    EVENT_START = 0
    EVENT_END = 30794385
    
elif ARM == "q":
    EVENT_COLUMN = "gain_event"
    EXCLUDE_COLUMN = "loss_event"
    EVENT_START = 80794385
    EVENT_END = 130794385

else:
    raise ValueError("Invalid value for ARM variable.")

In [4]:
cancer_types = {
    "brca": cptac.Brca,
    "colon": cptac.Colon,
    "hnscc": cptac.Hnscc,
    "lscc": cptac.Lscc,
    "luad": cptac.Luad,
    "ovarian": cptac.Ovarian
}

## Select the RNAs we're interested in

If we're looking at cis effects, we select RNAs within the event. If we're looking at trans effects, we select RNAs outside of the event.

In [5]:
selected_genes = cnvutils.\
get_event_genes(
    chrm=CHROMOSOME,
    event_start=EVENT_START,
    event_end=EVENT_END,
    cis_or_trans=CIS_OR_TRANS
)["Name"].\
drop_duplicates(keep="first")

def load_transcriptomics_tumor(dataset_func):
    return dataset_func().get_transcriptomics(tissue_type="tumor")

transcriptomics = {}

for cancer_type in cancer_types.keys():
    df = load_transcriptomics_tumor(cancer_types[cancer_type]).transpose()

    if df.index.nlevels == 1:
        df = df[df.index.isin(selected_genes)]
    else:
        df = df[df.index.isin(selected_genes, level="Name")]

    transcriptomics[cancer_type] = df

cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 11)


## Append Event Data

We now append the data from the event table that should have been created in a previous notebook.

In [6]:
has_event = dict()
for cancer_type in transcriptomics.keys():
    df = transcriptomics[cancer_type]
    df = df.transpose()
    event = pd.read_csv(f'{cancer_type}_has_event.tsv', sep='\t', index_col=0)
    if EXCLUDE_COLUMN:
        event.drop(EXCLUDE_COLUMN, axis=1, inplace=True)
    event.index.rename('Name')
    df = df.join(event)
    df = df.dropna(subset=[EVENT_COLUMN])
    has_event[cancer_type] = df[EVENT_COLUMN]
    transcriptomics[cancer_type] = df

## Run T-Tests

In [7]:
results_df = None
for cancer_type in transcriptomics.keys():
    prot_df = transcriptomics[cancer_type]
    results = cptac.utils.wrap_ttest(
        df=prot_df, 
        label_column=EVENT_COLUMN,
        correction_method="fdr_bh",
        return_all=True,
        quiet=True
    )   
    results.set_index('Comparison', inplace=True)
    if isinstance(results.index[0], tuple):
        results[['Name', f'{cancer_type}_Database_ID']] = pd.DataFrame(
            results.index.values.tolist(),
            index=results.index
        )
        results.set_index(['Name', f'{cancer_type}_Database_ID'], inplace=True)
    else:
        results.index.name='Name'
    results.rename(columns={'P_Value': f'{cancer_type}_pvalue'}, inplace=True)
    if results_df is None:
        results_df = results
    else:
        results_df = results_df.join(results)

## Append Difference Data

We want to collect one more piece of information about the data. We want to find the difference between the averages of the two groups. This will tell us if there is a positive or negative coorelation between the event and the proteomic data and can give us some indication of the strength of the coorelation. 

In [8]:
def get_diff(col, event):
    has_event = col[event]
    invert_list = [not x for x in event]
    no_event = col[invert_list]
    event_avg = has_event.mean()
    no_event_avg = no_event.mean()
    return event_avg - no_event_avg

In [9]:
diff_df = None
for cancer_type in transcriptomics.keys():
    df = transcriptomics[cancer_type]
    df = df.drop(EVENT_COLUMN, axis=1)
    results = df.apply(lambda x: get_diff(x, has_event[cancer_type]))
    df = pd.DataFrame(results)
    if isinstance(df.index[0], tuple):
        df[['Name', f'{cancer_type}_Database_ID']] = pd.DataFrame(df.index.values.tolist(), index=df.index)
        df.set_index(['Name', f'{cancer_type}_Database_ID'], inplace=True)
    else:
        df.index.name='Name'
    df.rename(columns={0: f'{cancer_type}_diff'}, inplace=True)
    if diff_df is None:
        diff_df = df
    else:
        diff_df = diff_df.join(df)

## Join the tables and save

We now join the difference table and the results table together. We also save the table to a tsv for use in future analyses. 

In [10]:
results_df = results_df.join(diff_df)

In [11]:
results_df

,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
Name,,,,,,,,,,,,
TOP1MT,2.605175e-09,4.401881e-07,0.021116,0.107717,2.427309e-02,0.079310,1.449963,0.803888,0.519419,0.393214,0.525443,8.955254
RRS1,2.605175e-09,5.490775e-05,0.009049,0.000031,6.712342e-06,NaN,1.501254,0.508240,0.448680,0.636500,0.728379,NaN
COPS5,3.194426e-08,6.425433e-04,0.000012,0.000016,1.838771e-08,0.256706,0.917840,0.504839,0.425694,0.443797,0.586322,20.906637
MTERFD1,2.318872e-07,NaN,NaN,NaN,NaN,0.062325,1.615228,NaN,NaN,NaN,NaN,7.810962
MAF1,2.668399e-07,2.183601e-05,0.111776,0.387288,6.103061e-03,0.800885,1.115379,0.478114,0.259653,0.173644,0.406493,32.036839
SNHG6,5.827528e-07,7.218737e-02,0.000492,0.122170,NaN,NaN,1.486062,0.359097,0.515207,0.394708,NaN,NaN
HSF1,7.146219e-07,5.194151e-06,0.419635,0.173826,8.691102e-03,0.016907,1.211482,0.627048,0.189284,0.265787,0.473446,10.826778
PABPC3,1.104049e-06,NaN,0.936632,0.222634,4.515763e-02,0.000580,1.085599,NaN,0.088959,0.389239,0.682020,82.904291
ZNF696,1.218214e-06,5.482622e-07,0.222269,0.226912,1.062001e-04,0.000205,1.159829,0.771332,0.245373,0.305868,0.741918,1.365811


In [12]:
# This will save the resulting table in the same directory as this notebook.
# Modify if you would like to save to a different location.
results_df.to_csv(f"{CHROMOSOME}{ARM}_{CIS_OR_TRANS}RNAeffects_ttest.tsv", sep='\t')